In [38]:
# Install required libraries
!pip install pandas numpy scipy

# Import libraries
import pandas as pd
import numpy as np
from scipy.stats import norm

# Load the dataset


# Replace with the path to your dataset
file_path = "/content/TRADES_CopyTr_90D_ROI.csv"  # Update this path
df = pd.read_csv(file_path)

# Inspect the dataset
print(df.head())
print(df.info())

              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB
None


In [42]:
import pandas as pd
import numpy as np
import ast

# Load the dataset
data = pd.read_csv('/content/TRADES_CopyTr_90D_ROI.csv')

# Inspect the dataset
print(data.head())
print(data.info())

# Handle missing values
data.dropna(subset=['Trade_History'], inplace=True)

# Convert the 'Trade_History' column from string to list of dictionaries
data['Trade_History'] = data['Trade_History'].apply(ast.literal_eval)

# Explode the 'Trade_History' column into individual rows
exploded_data = data.explode('Trade_History')

# Convert the dictionaries into separate columns
trade_data = pd.json_normalize(exploded_data['Trade_History'])

# Combine the exploded data with the original Port_IDs
trade_data['Port_IDs'] = exploded_data['Port_IDs'].values

# Inspect the new DataFrame
print(trade_data.head())

# Convert relevant columns to numeric
trade_data['realizedProfit'] = pd.to_numeric(trade_data['realizedProfit'], errors='coerce')
trade_data['fee'] = pd.to_numeric(trade_data['fee'], errors='coerce')
trade_data['quantity'] = pd.to_numeric(trade_data['quantity'], errors='coerce')

# Group by Port_IDs and calculate metrics
def calculate_metrics(x):
    total_investment = x.loc[x['side'] == 'BUY', 'quantity'].sum()
    net_profit = x['realizedProfit'].sum() - x['fee'].sum()

    # Handle division by zero for ROI
    roi = (net_profit / total_investment * 100) if total_investment != 0 else 0

    # Handle division by zero for Sharpe Ratio
    sharpe_ratio = (x['realizedProfit'].mean() / x['realizedProfit'].std()) if x['realizedProfit'].std() != 0 else 0

    # Handle division by zero for MDD
    cumulative_pnl = x['realizedProfit'].cumsum()
    peak = cumulative_pnl.max()
    trough = cumulative_pnl.min()
    mdd = ((peak - trough) / peak) if peak != 0 else 0

    return pd.Series({
        'ROI': roi,
        'PnL': net_profit,
        'Sharpe Ratio': sharpe_ratio,
        'MDD': mdd,
        'Win Rate': (x['realizedProfit'] > 0).mean() * 100,
        'Win Positions': (x['realizedProfit'] > 0).sum(),
        'Total Positions': len(x)
    })

metrics = trade_data.groupby('Port_IDs').apply(calculate_metrics).reset_index()

# Rank accounts based on a composite score
weights = {
    'ROI': 0.3,
    'PnL': 0.25,
    'Sharpe Ratio': 0.2,
    'MDD': 0.15,
    'Win Rate': 0.1
}

metrics['Composite Score'] = (
    metrics['ROI'] * weights['ROI'] +
    metrics['PnL'] * weights['PnL'] +
    metrics['Sharpe Ratio'] * weights['Sharpe Ratio'] +
    (1 - metrics['MDD']) * weights['MDD'] +  # Lower MDD is better
    metrics['Win Rate'] * weights['Win Rate']
)

metrics = metrics.sort_values(by='Composite Score', ascending=False)

# Save the results
metrics.to_csv('account_metrics.csv', index=False)

# Top 20 accounts
top_20 = metrics.head(20)
print(top_20)

              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB
None
            time    symbol side      price       fee feeAsset    quantity  \
0  1718899656000   SOLUSDT  BUY  132.53700 -0.994027     USDT  1988.05500   
1  1718899618000  DOGEUSDT  BUY    0.12182 -0.279796     USDT  1398.980

<ipython-input-42-80945898458d>:62: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = trade_data.groupby('Port_IDs').apply(calculate_metrics).reset_index()


In [44]:
top_20.head(20)

,Port_IDs,ROI,PnL,Sharpe Ratio,MDD,Win Rate,Win Positions,Total Positions,Composite Score
96,4020204877254599680,0.733446,83347.153462,0.060748,1.106559,32.942149,1993.0,6050.0,20840.298780
75,3999240873283311617,1.141374,44446.918627,0.227505,0.997025,52.321981,2366.0,4522.0,11117.350215
98,4021669203289716224,0.859339,27963.774511,0.068251,0.998458,31.391201,528.0,1682.0,6994.354431
50,3960874214179953664,0.505165,21527.023137,0.162948,1.057843,51.568381,1233.0,2391.0,5387.088085
107,4028701921959171840,0.896220,19753.162333,0.275724,1.000000,54.294032,373.0,687.0,4944.043997
17,3907081197088384000,2.267194,18638.536459,0.222729,1.000000,61.397148,2540.0,4137.0,4666.498534
48,3956076827719377409,2.091124,17430.479532,0.099528,1.000000,51.557712,2019.0,3916.0,4363.422897
62,3986814617275053313,1.751192,16969.210989,0.329787,1.000000,78.221722,2780.0,3554.0,4250.716234
99,4022565861939831809,0.557467,16866.053765,0.299602,1.000000,52.765957,372.0,705.0,4222.017197
5,3788465932399412480,1.739763,14687.797494,0.152912,0.998232,44.607364,1369.0,3069.0,3676.962886
